In [6]:
import numpy as np
import parmed as pmd

np.set_printoptions(precision=4, suppress=True)

def euler_to_matrix(psi, theta, phi):
    psi, theta, phi = np.radians(psi), np.radians(theta), np.radians(phi)
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    R_y = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    R_z = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    rotation_matrix = np.dot(R_z, np.dot(R_y, R_x))
    return rotation_matrix

def compute_cent(fname):
    instruc = pmd.load_file(fname)
    coords = instruc.coordinates

    if len(instruc.residues) != 1:
        print('Error: More than one residue located')
        return

    res = instruc.residues[0]
    IDAtoms = ['NA', 'NB', 'NC', 'ND']
    IDNDX = np.zeros((len(IDAtoms),), dtype=int)
    foundAt = np.zeros((len(IDAtoms),), dtype=int)

    for n in range(0, len(IDAtoms)):
        name = IDAtoms[n]
        for at in res.atoms:
            if at.name == name:
                foundAt[n] = 1
                IDNDX[n] = at.idx
    
    if np.sum(foundAt) == len(IDAtoms):
        xA, xB, xC, xD = coords[IDNDX, :]
        cent = (xA+xC)/2
        return cent

def shift_molecule(coordinates, d_x, d_y, d_z):
    translation = coordinates + np.array([d_x, d_y, d_z])
    return translation

def rotate_to_global(fname):
    instruc = pmd.load_file(fname)
    coords = instruc.coordinates

    if len(instruc.residues) != 1:
        print('Error: More than one residue located')
        return

    res = instruc.residues[0]
    IDAtoms = ['NA', 'NB', 'NC', 'ND']
    IDNDX = np.zeros((len(IDAtoms),), dtype=int)
    foundAt = np.zeros((len(IDAtoms),), dtype=int)

    for n in range(0, len(IDAtoms)):
        name = IDAtoms[n]
        for at in res.atoms:
            if at.name == name:
                foundAt[n] = 1
                IDNDX[n] = at.idx
    
    if np.sum(foundAt) == len(IDAtoms):
        xA, xB, xC, xD = coords[IDNDX, :]
        vCA = (xA - xC)
        vCA /= np.linalg.norm(vCA)

        vDB = (xB - xD)
        vDB /= np.linalg.norm(vDB)

        vnorm = np.cross(vCA, vDB)
        vnorm /= np.linalg.norm(vnorm)
        
        z_axis_local = -vnorm
        y_axis_local = vCA
        x_axis_local = np.cross(vCA, -vnorm)
        x_axis_local /= np.linalg.norm(x_axis_local)
        
        r12_x = xx = x_axis_local[0]
        r12_y = yx = x_axis_local[1]
        r12_z = zx = x_axis_local[2]
        
        r13_x = xy = y_axis_local[0]
        r13_y = yy = y_axis_local[1]
        r13_z = zy = y_axis_local[2]
        
        cross_x = xz = z_axis_local[0]
        cross_y = yz = z_axis_local[1]
        cross_z = zz = z_axis_local[2]
        
        rotmat_local_to_global = np.array([[xx, yx, zx],[xy,yy,zy],[xz,yz,zz]])

        return rotmat_local_to_global

def rotate_pigments(fname, d_z, d_y, d_x, d_psi, d_theta, d_phi):
    instruc = pmd.load_file(fname)
    coords = instruc.coordinates
    cent = compute_cent(fname)
    if len(instruc.residues) != 1:
        print('Error: More than one residue located')
        return

    res = instruc.residues[0]
    struc1 = pmd.structure.Structure()
    struc2 = pmd.structure.Structure()
    coords1 = []  # new molecule
    coords2 = []  # original molecule
    rotation_matrix = rotate_to_global(fname)
    rotation_matrix_euler = euler_to_matrix(d_psi,d_theta,d_phi) #add loop and adjust angles 
    #add shift function and shift molecule
    
    # Rotate and center molecule 1
    for at in res.atoms:
        oldx = coords[at.idx, :]
        newx1 = oldx.copy()-cent 
        newx2 = oldx.copy()-cent
        # Apply the rotation to the coordinates of molecule 1
        newx1 = np.dot(rotation_matrix, newx1)
        newx1 = np.dot(rotation_matrix_euler,newx1)
        newx1 = shift_molecule(newx1, d_x, d_y, d_z)
        struc1.add_atom(pmd.topologyobjects.Atom(name=at.name, atomic_number=at.atomic_number), res.name, 1, chain='A')
        coords1.append(newx1)

        # Save the original coordinates of molecule 2
        newx2 = np.dot(rotation_matrix, newx2)
        struc2.add_atom(pmd.topologyobjects.Atom(name=at.name, atomic_number=at.atomic_number), res.name, 2, chain='B')
        coords2.append(newx2)



    # Combine struc1 and struc2
    combined_struc = struc1 + struc2

    # Write the combined structure to a PDB file
    combined_struc.coordinates = np.vstack([np.array(coords1), np.array(coords2)])
    combined_struc.write_pdb(
        'shift_z_' + '{:.1f}'.format(d_z) + '_shift_y_' + '{:.1f}'.format(
            d_y) + '_shift_x_' + '{:.1f}'.format(d_x) + '_psi_' + '{:.0f}'.format(d_psi) + '_theta_' + '{:.0f}'.format(
            d_theta) + '_phi_' + '{:.0f}'.format(d_phi) + '.pdb')


    
for j in range(2, 3): #d_z
    for k in range(0,1): #d_y
        for l in range(0,1): #d_x
            for m in range(40, 180, 10): #psi - can range from -180 to 180
                for n in range(-90, 90, 10): #theta - can range from -90 to 90
                    for o in range(-180, 180, 10): #phi - can range from -180 to 180
                        rotate_pigments('0_monomerMG.pdb', j, k, l, m, n, o) 